# T5 Model Demo
Author: Prakhar Mishra

Connect me on [LinkedIn](https://www.linkedin.com/in/prakhar21) \\
See more of my work at [GitHub](https://www.github.com/prakhar21) \\
Read by [Blogs](https://www.prakhartechviz.blogspot.com)

In [115]:
!pip install transformers --upgrade
#transformers >= 2.8.0

Requirement already up-to-date: transformers in /usr/local/lib/python3.6/dist-packages (2.8.0)


# Using Main Class

In [0]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import time

model = T5ForConditionalGeneration.from_pretrained('t5-small')  #tiny-base
tokenizer = T5Tokenizer.from_pretrained('t5-small')  #tiny-base

start_time = time.time()

def summarization_infer(text, max=50):
  preprocess_text = text.replace("\n", " ").strip()
  t5_prepared_Text = "summarize: "+preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")

  summary_ids = model.generate(tokenized_text, min_length=30, max_length=max, top_k=100, top_p=0.8) #top-k top-p sampling strategy
  output = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  end_time = time.time()
  print (f'Time taken : {end_time-start_time}')
  return output

def translation_infer(text, max=50):
  preprocess_text = text.replace("\n", " ").strip()
  t5_prepared_Text = "translate English to German: "+preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")

  translation_ids = model.generate(tokenized_text, min_length=10, max_length=50, early_stopping=True, num_beams=2)
  output = tokenizer.decode(translation_ids[0], skip_special_tokens=True)
  end_time = time.time()
  print (f'Time taken : {end_time-start_time}')
  return output

def grammatical_acceptibility_infer(text):
  preprocess_text = text.replace("\n", " ").strip()
  t5_prepared_Text = "cola sentence: "+preprocess_text
  tokenized_text = tokenizer.encode(t5_prepared_Text, return_tensors="pt")

  grammar_ids = model.generate(tokenized_text, min_length=1, max_length=3)
  output = tokenizer.decode(grammar_ids[0], skip_special_tokens=True)
  end_time = time.time()
  print (f'Time taken : {end_time-start_time}')
  return output

# Using PyTorch Pipelines
Newly introduced in transformers v2.3.0, pipelines provides a high-level, easy to use, API for doing inference over a variety of downstream-tasks. Read [more practical documentation](https://www.kaggle.com/funtowiczmo/hugging-face-transformers-how-to-use-pipelines)

In [147]:
from transformers import pipeline

summarization_pipeline = pipeline(task='summarization', model="t5-small") 
output = summarization_pipeline(text, min_length=30, max_length=50, top_k=100, top_p=0.8)
print (output)

[{'summary_text': "india's novel coronavirus count has increased to 21,700, according to the latest ministry of health and family welfare data . the death toll due to the virus is 686 after 34 more patients succumbed to the highly"}]


# Making Flask API

In [150]:
from flask import Flask, request

app = Flask(__name__)

@app.route('/infer', methods=['POST'])
def infer():
  args = request.args['task']
  text = request.args['text']
  if args=='summarize':
    return summarization_infer(text)
  elif args=='translation':
    return translation_infer(text)
  else:
    return grammatical_acceptibility_infer(text)

if __name__=='__main__':
  app.run(host='0.0.0.0', port=5555, debug=False, threaded=True)
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5555/ (Press CTRL+C to quit)


In [0]:
text ="""
With 1,229 fresh cases in the last 24 hours, India's novel coronavirus count has increased to 21,700, according to the latest Ministry of Health and Family Welfare data. Along with that, the death toll due to the virus has increased to 686 after 34 more patients succumbed to the highly contagious disease since yesterday, it said. So far, India has 16,689 active cases. There are also 77 foreign nationals who are affected by the virus, the ministry said. Apart from that, there are at least 4,324 patients who have been discharged or cured from the highly contagious disease and one has migrated from the country. Speaking at the press briefing today, Lav Agarwal, Joint Secretary, Health Ministry, said, "As on today, we have 12 districts that did not have a fresh case in the last 28 days or more. There are now 78 districts (23 States/UTs) that has not reported any fresh cases during the last 14 Days." However, he also said that the increase in the number of coronavirus cases in the country is "more or less linear, not exponential." According to the Thursday morning data of health ministry, 4,257 Covid-19 patients have been cured so far, bring the recovery rate to is 19.89% as of now, said Aggrawal. He also added, "We have been able to cut virus transmission, minimise spread of COVID-19 in 30 days of lockdown."
"""

In [161]:
summarization_infer(text)

Time taken : 158.3986692428589


"india's novel coronavirus count has increased to 21,700, according to the latest ministry of health and family welfare data. the death toll due to the virus has increased to 686 after 34 more patients succumbed to"